In [1]:
import sys
sys.path.append("../")
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import keras2onnx
import onnxruntime
print(f"Tensoflow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
from models import endpoint_in_model
from losses import euclidean_distance_loss,endpoint_loss
from data_processing.data_loader import dataset_loader
from config import params

from time import time
import keract



os.environ['CUDA_VISIBLE_DEVICES']="1"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Tensoflow Version: 2.3.0
Keras Version: 2.4.0
Num GPUs Available:  1


In [3]:
"""
dataset_dir = '/netpool/work/gpu-3/users/malyalasa/New_folder/tf_records'
ds_loader = dataset_loader(tfrec_dir=dataset_dir, batch_size=32)
ds_train, ds_valid, ds_test = ds_loader.build_dataset()
"""
dataset_dir = '/netpool/work/gpu-3/users/malyalasa/New_folder/tf_records'
ds_loader = dataset_loader(tfrec_dir=params.get("dataset_dir"), batch_size=32)
#ds_train, ds_valid, ds_test = ds_loader.build_dataset()
ds_train, ds_valid, ds_test = ds_loader.build_scenario_dataset(consider_scenes=10,no_train_scene=8,no_valid_scene=1,no_test_scene=1)


Scenarios:['/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario1', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario2', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario3', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario4', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario5', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario_ph_11', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario_ph_12', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario_ph_13', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario_ph_14', '/bigpool/projects/yao_SCANGAN360/New_Folder/tf_records/scenario_ph_15']
Building Dataset.......

Number of records, Train files:610, validation files:109, Test Files:98
((TensorSpec(shape=(None, 1536, 1536), dtype=tf.int8, name=None),
  TensorSpec(shape=(None, None), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 25, 2), dtype=tf.float32, name=N

In [5]:
# load model wwith CustomMaskLayer
pp_model = tf.keras.models.load_model('/netpool/work/gpu-3/users/malyalasa/New_folder/wandb/latest-run/files/model-best.h5',compile=False,custom_objects={'CustomMaskLayer':endpoint_in_model.CustomMaskLayer})

#pp_model = tf.keras.models.load_model('/netpool/work/gpu-3/users/malyalasa/New_folder/wandb/run-20210709_111150-1dpjz991/files/model-best.h5',compile=False,custom_objects={'CustomMaskLayer':endpoint_in_model.CustomMaskLayer})


pp_model.summary()
# Compile the model
opt = tf.keras.optimizers.Adam(learning_rate=0.02)
pp_model.compile(
        optimizer=opt, 
        loss=[euclidean_distance_loss,endpoint_loss],
        loss_weights=params.get("loss_weights"), metrics=params.get("metric"))

# evaluate
test_loss,test_accuracy = pp_model.evaluate(ds_test)


OSError: SavedModel file does not exist at: /netpool/work/gpu-3/users/malyalasa/New_folder/wandb/latest-run/files/model-best.h5/{saved_model.pbtxt|saved_model.pb}

In [5]:
data = np.load('/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/dataset/sample_gridmap.npz')
input_sample = [data[f'{file}'] for file in data.files]

#print(len(input_sample))
pp_model.predict(input_sample)

array([[[ 352.41058, -667.99963],
        [ 338.91028, -594.2906 ],
        [ 335.09494, -517.82385],
        [ 356.2013 , -441.23682],
        [ 392.94614, -374.8775 ],
        [ 414.2722 , -351.64706],
        [ 416.53925, -379.44757],
        [ 413.40344, -419.11337],
        [ 405.43573, -461.00156],
        [ 397.44992, -503.15634],
        [ 389.1074 , -543.16235],
        [ 377.92828, -585.3933 ],
        [ 363.03876, -624.7076 ],
        [ 355.1357 , -644.1888 ],
        [ 353.25412, -654.2156 ],
        [ 350.88242, -662.4044 ],
        [ 348.79312, -667.7455 ],
        [ 348.62888, -672.538  ],
        [ 348.3208 , -675.0437 ],
        [ 347.99033, -676.7255 ],
        [ 347.80084, -676.9148 ],
        [ 348.0718 , -676.81464],
        [ 347.82907, -677.1928 ],
        [ 347.8195 , -677.03284],
        [ 347.83392, -677.35364]]], dtype=float32)

In [6]:
activations = keract.get_activations(pp_model, input_sample, auto_compile=True)

In [6]:
activations_dir = '/netpool/work/gpu-3/users/malyalasa/New_folder/rosbag2numpy/logging/after_norm_and_coordconv'

keract.display_activations(activations, cmap=None, save=True, directory=activations_dir, data_format='channels_last', fig_size=(10, 10), reshape_1d_layers=False)


input_1 (1, 1536, 1536, 1) 
Grid_origin_res (1, 3) 
Left_boundary (1, 25, 2) 
Right_boundary (1, 25, 2) 
Car_loc (1, 3) 
Initial_path (1, 25, 2) 
conv2d (1, 767, 767, 16) 
leaky_re_lu (1, 767, 767, 16) 
batch_normalization (1, 767, 767, 16) 
average_pooling2d (1, 191, 191, 16) 
conv2d_1 (1, 189, 189, 32) 
leaky_re_lu_1 (1, 189, 189, 32) 
batch_normalization_1 (1, 189, 189, 32) 
conv2d_2 (1, 189, 189, 8) 
conv2d_3 (1, 189, 189, 1) 
flatten (1, 35721) 
reshape (1, 50) 
reshape_1 (1, 50) 
reshape_2 (1, 50) 
concatenate (1, 6) 
concatenate_1 (1, 35877) 
dense (1, 16) 
batch_normalization_2 (1, 16) 
dropout (1, 16) 
dense_1 (1, 50) 
custom_mask_layer (1, 25, 2) 
dropout_1 (1, 50) 
reshape_3 (1, 50) 
add (1, 50) 
dense_2 (1, 50) 
reshape_4 (1, 25, 2) 
